In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sivarazadi/wikiart-art-movementsstyles")

print("Path to dataset files:", path)

Mounting files to /kaggle/input/wikiart-art-movementsstyles...
Path to dataset files: /kaggle/input/wikiart-art-movementsstyles


In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

IMG_SIZE = 224
BATCH_SIZE = 32

data_dir = "/kaggle/input/wikiart-art-movementsstyles"

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)

val_data = datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)

Found 34004 images belonging to 13 classes.
Found 8496 images belonging to 13 classes.


In [ ]:
base = MobileNetV2(
    weights=None,          
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

x = GlobalAveragePooling2D()(base.output)
x = Dense(256, activation='relu')(x)
output = Dense(train_data.num_classes, activation='softmax')(x)

model = Model(base.input, output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=3
)